In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score

def naiveBayesTrain(X,Y,data):
    lambd = 1
    dictY = Y.groupby(['label']).size().reset_index(name='Size')
    labelCount = dictY.shape[0]
    labelSize = dictY.sum().values[1]
    dictY['probability'] = dictY['Size'].apply(lambda x: (x + lambd) / (labelSize + labelCount * lambd))
    featureDict = {}
    for key in X.columns.tolist():
        temp = data.groupby([key,'label']).size().reset_index(name='Size').pivot(key,'label','Size').fillna(0)
        for label_key in temp.columns.tolist():
            temp[label_key] = temp[label_key].apply(lambda x: (x + lambd) / (temp[label_key].sum() + temp.shape[0] * lambd))
        featureDict[key] = temp
    return featureDict,dictY

def naiveBayesPredict(X_pre,featureDict,labelList,dictY):
    Y_pre = []
    columnsList = X_pre.columns.tolist()
    for i in range(X_pre.shape[0]):
        resultList = []
        feaList = X_pre.iloc[i].reset_index(name='feaValues').rename(columns={'index':'feaName'})
        for labelKey in labelList:
            temProbability = 0
            for columnkey in columnsList:
                feaValue = feaList[feaList['feaName'] == columnkey].values[0][1]
                temProbability += math.log(featureDict[columnkey].loc[feaValue][labelKey])
            resultList.append(temProbability + math.log(dictY[dictY['label'] == labelKey]['probability'].values[0]))
        Y_pre.append(labelList[resultList.index(max(resultList))])
    return pd.DataFrame(Y_pre,columns=['label'])

if __name__ == '__main__':
    data = pd.read_csv('/Users/world/课程资料/数据分析工具实践/第三次作业/第3次作业/贝叶斯/数据/nursery.data',header=None)
    data.columns=['parents','has_nurs','form','children','housing','finance','social','health','label']
    dataPre = data.sample(n=100).reset_index(drop=True)
    X = data.drop(['label'],axis=1)
    Y = pd.DataFrame(data['label'])
    labelList = Y.groupby(['label']).count().index.tolist()
    featureDict, dictY = naiveBayesTrain(X,Y,data)
    X_pre = dataPre.drop(['label'],axis=1)
    Y_pre= naiveBayesPredict(X_pre,featureDict,labelList,dictY)
    Y_true = pd.DataFrame(dataPre['label'])
    temp = pd.concat([Y_pre,Y_true],axis=1)
    temp.columns = ['Y_pre','Y_true']
    print("分类准确率： ", accuracy_score(Y_true, Y_pre))
    print(temp)

分类准确率：  0.92
         Y_pre      Y_true
0   spec_prior  spec_prior
1     priority    priority
2   spec_prior  spec_prior
3     priority    priority
4   spec_prior  spec_prior
5     priority    priority
6   spec_prior  spec_prior
7    not_recom   not_recom
8     priority    priority
9   spec_prior  spec_prior
10    priority    priority
11  spec_prior  spec_prior
12    priority    priority
13    priority    priority
14    priority    priority
15  spec_prior  spec_prior
16  spec_prior  spec_prior
17   not_recom   not_recom
18   not_recom   not_recom
19  spec_prior  spec_prior
20    priority    priority
21  spec_prior    priority
22   not_recom   not_recom
23   not_recom   not_recom
24    priority    priority
25   not_recom   not_recom
26    priority    priority
27   not_recom   not_recom
28   not_recom   not_recom
29  spec_prior  spec_prior
..         ...         ...
70   not_recom   not_recom
71   not_recom   not_recom
72  spec_prior  spec_prior
73   not_recom   not_recom
74  spec_prior 